<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/039_GAME_Agent_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This new agent implementation using the `Agent` class is a **modularized and extensible version** of the earlier, more procedural agent we built manually.

Here's a breakdown of how they differ and how the new version aligns with the GAME framework:

---

## 🧱 Structural Differences

| Aspect                      | ✅ Modular Agent (New)                                                      | 🧪 Manual Agent (Old)                                       |
| --------------------------- | -------------------------------------------------------------------------- | ----------------------------------------------------------- |
| **Design Pattern**          | Uses the GAME framework (Goals, Actions, Memory, Environment)              | Procedural logic, tool calls, and routing are all hardcoded |
| **Extensibility**           | Easy to add new actions, goals, or swap environments                       | Requires editing multiple parts of the code manually        |
| **Memory**                  | Automatically tracked and passed into the prompt                           | Memory was minimal or simulated manually                    |
| **Environment Abstraction** | Encapsulated in a class                                                    | Direct function calls                                       |
| **Function Calling**        | Uses OpenAI tool calling formally via `AgentFunctionCallingActionLanguage` | Tools were manually routed and parsed                       |

---

## 🧠 Similarities

Despite the structural upgrades, both agents:

* Use tool definitions based on user intent
* Accept user input and call an LLM to decide the next step
* Can simulate a loop through tool execution and reasoning
* Aim to produce structured tool calls in response to natural input

---

## 🔄 Functional Equivalents

| Concept         | Manual Agent                          | GAME Agent                                                     |
| --------------- | ------------------------------------- | -------------------------------------------------------------- |
| Tool router     | `tool_router()`                       | `ActionRegistry.get_action()` + `Environment.execute_action()` |
| Tool schema     | JSON tool definitions                 | `Action` dataclass w/ name, parameters, terminal flag          |
| LLM call        | `client.chat.completions.create(...)` | `generate_response()` injected dependency                      |
| Handling result | Manual parsing                        | `agent.run()` loop w/ result tracking                          |
| Memory          | List of message dicts                 | `Memory` class w/ `.add_memory()` and retrieval                |

---

## 🧩 Key Advantages of the GAME Version

1. **Reusable across agents**: You can use the same `Agent` class for many agents just by swapping goals and actions.
2. **Centralized configuration**: Actions are clearly defined and typed using `Action`.
3. **Memory persistence**: Automatically logs thoughts and actions step by step.
4. **Support for `terminate`**: Enables clean exits with final summaries.
5. **Cleaner control flow**: You don’t have to manually write tool call parsing logic or handle fallback behavior.

---

## ✅ Bottom Line

You're still building an agent, but:

* ✅ Now it’s organized around modular classes (`Agent`, `ActionRegistry`, `Memory`, `Environment`)
* ✅ It’s aligned with the **GAME** framework
* ✅ You can **scale, test, and swap pieces** without rewriting the whole thing


In [ ]:
def main():
    # Define the agent's goals
    goals = [
        Goal(
            priority=1,
            name="Explore Files",
            description="Explore files in the current directory by listing and reading them"
        ),
        Goal(
            priority=2,
            name="Terminate",
            description="Terminate the session when tasks are complete with a helpful summary"
        )
    ]

    # Define tool functions
    def list_files() -> List[str]:
        """List files in the current directory."""
        return os.listdir(".")

    def read_file(file_name: str) -> str:
        """Read a file's contents."""
        try:
            with open(file_name, "r") as file:
                return file.read()
        except FileNotFoundError:
            return f"Error: {file_name} not found."
        except Exception as e:
            return f"Error: {str(e)}"

    def terminate(message: str) -> str:
        """Terminate the agent loop and provide a summary message."""
        return message

    # Create action registry and register actions
    action_registry = ActionRegistry()

    action_registry.register(Action(
        name="list_files",
        function=list_files,
        description="Returns a list of files in the directory.",
        parameters={},
        terminal=False
    ))

    action_registry.register(Action(
        name="read_file",
        function=read_file,
        description="Reads the content of a specified file in the directory.",
        parameters={
            "type": "object",
            "properties": {
                "file_name": {"type": "string"}
            },
            "required": ["file_name"]
        },
        terminal=False
    ))

    action_registry.register(Action(
        name="terminate",
        function=terminate,
        description="Terminates the conversation. Prints the provided message for the user.",
        parameters={
            "type": "object",
            "properties": {
                "message": {"type": "string"},
            },
            "required": ["message"]
        },
        terminal=True
    ))

    # Define the agent language and environment
    agent_language = AgentFunctionCallingActionLanguage()
    environment = Environment()

    # Create the agent
    file_explorer_agent = Agent(
        goals=goals,
        agent_language=agent_language,
        action_registry=action_registry,
        generate_response=generate_response,
        environment=environment
    )

    # Run the agent
    user_input = input("What would you like me to do? ")
    final_memory = file_explorer_agent.run(user_input, max_iterations=10)

    # Print the termination message (if any)
    for item in final_memory.get_memories():
        print(f"\nMemory: {item['content']}")

if __name__ == "__main__":


# 🧠 Benefits of Converting to the GAME Framework

By adopting the **GAME** (Goals, Actions, Memory, Environment) framework for agent design, you gain several powerful benefits:

---

## ✅ Key Differences & Advantages

### 🧱 Better Organization

Each component has a single responsibility:

* **Goals** define purpose
* **Actions** define what the agent can do
* **Memory** stores context
* **Environment** runs the actions

---

### 🔁 Reusability

You can:

* Reuse **actions** across agents
* Swap **environments** (local, API-based, remote, etc.)
* Test different **goal sets** with the same tools

---

### ➕ Extensibility

Easily add:

* New tools (actions)
* More advanced memory strategies
* Multi-step planning logic

---

### 📦 Standard Interface

All agents use a consistent `Agent` class and method signatures:

```python
final_memory = agent.run(user_input)
```

---

### 🧠 Memory Management

The framework:

* Automatically stores inputs and results
* Maintains full history
* Ensures relevant context is passed to each loop

---

## 💬 Real Usage Example

### User Input:

> "Tell me what Python files are in this directory and summarize how they fit together."

---

### 🔄 Agent Loop In Action:

```json
{"tool_name": "list_files", "args": {}}
```

✅ **Action Result**:

```json
{
  "tool_executed": true,
  "result": ["file1.py", "file2.py", "main.py"],
  "timestamp": "2025-03-02T12:34:56+0000"
}
```

---

```json
{"tool_name": "read_file", "args": {"file_name": "file1.py"}}
```

✅ **Action Result**:

```json
{
  "tool_executed": true,
  "result": "# This is file1.py\n\ndef hello_world():\n    print(\"Hello, World!\")",
  "timestamp": "2025-03-02T12:34:58+0000"
}
```

---

### ✅ Final Action

```json
{"tool_name": "terminate", "args": {
  "message": "I've explored all Python files. file1.py has a hello_world function, file2.py implements a calculator class, and main.py uses both."}
}
```

---

## 🚀 Summary

This structured, modular approach makes your agents:

* **Smarter** (can reason over context)
* **Safer** (fewer parsing bugs or ambiguous tool calls)
* **Scalable** (easy to grow as functionality increases)
* **Maintainable** (each part is clear and testable)



Let’s walk through each structural difference **one at a time**, with clear explanations and examples from both the **Modular Agent** and the **Manual Agent** you've worked on.

---

### 1. ✅ **Design Pattern**

| Modular Agent (New)                                                          | Manual Agent (Old)                                        |
| ---------------------------------------------------------------------------- | --------------------------------------------------------- |
| Uses the **GAME** framework: `Goals`, `Actions`, `Memory`, and `Environment` | Everything lives in a flat script with hardcoded behavior |

#### 🔍 Explanation:

* **Modular Agent** has clearly separated components:

  * `Goal`: What is the agent trying to achieve?
  * `Action`: Registered tools with formal schemas
  * `Memory`: All past responses and tool results
  * `Environment`: Executes tool logic in isolation

* **Manual Agent**:

  * You have `tool_router()` manually matching names
  * All logic happens in one loop; no separation of concerns

#### ✅ Why it matters:

Structured design helps you debug, extend, and test parts individually. For example, you can swap out the environment for a GitHub Actions-based runner without touching the agent logic.

---

### 2. ✅ **Extensibility**

| Modular Agent (New)                                   | Manual Agent (Old)                                                    |
| ----------------------------------------------------- | --------------------------------------------------------------------- |
| Add a new action with `action_registry.register(...)` | Must update `tool_router`, schema list, and response parsing manually |

#### 🔍 Explanation:

* In the modular setup, adding a new tool is as easy as:

  ```python
  action_registry.register(Action(
      name="summarize_file",
      function=summarize_file,
      ...
  ))
  ```

* In the manual agent:

  * You'd define the tool function
  * Manually add a `tool_schema` entry
  * Update `tool_router()` to handle it
  * Adjust parsing logic to route and return results

#### ✅ Why it matters:

This saves time and reduces human error, especially as your agent grows from 3 tools to 30+.

---

### 3. ✅ **Memory**

| Modular Agent (New)                        | Manual Agent (Old)                                       |
| ------------------------------------------ | -------------------------------------------------------- |
| `Memory` is a class that tracks every step | Only the user input or last result is stored (if at all) |

#### 🔍 Explanation:

* Modular Agent:

  ```python
  memory.add_memory({"type": "user", "content": user_input})
  memory.add_memory({"type": "assistant", "content": response})
  ```

* Manual Agent:

  ```python
  messages = [{"role": "user", "content": user_input}]
  # No long-term memory between calls
  ```

#### ✅ Why it matters:

Memory enables the agent to **reason over time**, avoid repeating itself, and build on prior results. This becomes critical for multi-step tasks.

---

### 4. ✅ **Environment Abstraction**

| Modular Agent (New)                                             | Manual Agent (Old)                    |
| --------------------------------------------------------------- | ------------------------------------- |
| Uses an `Environment()` class with an `execute_action()` method | Functions are called directly by name |

#### 🔍 Explanation:

* Modular Agent:

  ```python
  result = environment.execute_action(action, invocation["args"])
  ```

* Manual Agent:

  ```python
  result = tool_router(tool_name, args)
  ```

#### ✅ Why it matters:

The environment can later:

* Log action execution
* Simulate execution in tests
* Run in cloud / serverless contexts

And you don’t have to change the agent logic to support those.

---

### 5. ✅ **Function Calling**

| Modular Agent (New)                       | Manual Agent (Old)                             |
| ----------------------------------------- | ---------------------------------------------- |
| Uses OpenAI’s structured function calling | Manually parsed JSON or tool\_calls if present |

#### 🔍 Explanation:

* Modular Agent:

  ```python
  agent_language = AgentFunctionCallingActionLanguage()
  # All parsing and schema validation is handled internally
  ```

* Manual Agent:

  ```python
  if response.choices[0].message.tool_calls:
      tool_name = ...
      args = json.loads(...)
  ```

#### ✅ Why it matters:

Letting OpenAI handle schema validation and argument generation means:

* Fewer errors
* Less code
* No manual JSON parsing

